In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Mon Oct 23 22:40:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -q \
autodistill \
autodistill-grounded-sam \
autodistill-yolov8 \
supervision==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.0/527.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.9 MB/s eta 0:00:00


In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [5]:
IMAGE_DIR_PATH="/content/drive/Shareddrives/extradrive/Final_dataset/battery"

In [6]:
import supervision as sv

image_paths=sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["jpg"]
)

print('image count: ', len(image_paths))

image count:  860


In [7]:

SAMPLE_SIZE=50
SAMPLE_GRID_SIZE=(10,5)
SAMPLE_PLOT_SIZE=(50,50)

In [8]:
from autodistill.detection import CaptionOntology

ontology=CaptionOntology({
    "battery":"battery"
})

In [9]:
DATASET_DIR_PATH="/content/drive/Shareddrives/extradrive/Final_dataset/battery/F_battery_dataset"

In [10]:
from autodistill_grounded_sam import GroundedSAM

base_model = GroundedSAM(ontology=ontology)
dataset = base_model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".jpg",
    output_folder=DATASET_DIR_PATH)

trying to load grounding dino directly
downloading dino model weights


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Labeling /content/drive/Shareddrives/extradrive/Final_dataset/battery/battery101.jpg:   0%|          | 0/860 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:905: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Labeling /content/drive/Shareddrives/extradrive/Final_dataset/batt

Labeled dataset created - ready for distillation.


In [11]:
ANNOTATIONS_DIRECTORY_PATH = "/content/drive/Shareddrives/extradrive/Final_dataset/battery/F_battery_dataset/train/labels"
IMAGES_DIRECTORY_PATH = "/content/drive/Shareddrives/extradrive/Final_dataset/battery/F_battery_dataset/train/images"
DATA_YAML_PATH = "/content/drive/Shareddrives/extradrive/Final_dataset/battery/F_battery_dataset/data.yaml"

In [12]:
import supervision as sv

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=IMAGES_DIRECTORY_PATH,
    annotations_directory_path=ANNOTATIONS_DIRECTORY_PATH,
    data_yaml_path=DATA_YAML_PATH)

len(dataset)

688

In [ ]:
import supervision as sv

for i in range(1,18):
  image_names = list(dataset.images.keys())[SAMPLE_SIZE*(i-1):SAMPLE_SIZE*i]

  mask_annotator = sv.MaskAnnotator()
  box_annotator = sv.BoxAnnotator()

  images = []
  for image_name in image_names:
      image = dataset.images[image_name]
      annotations = dataset.annotations[image_name]
      labels = [
          dataset.classes[class_id]
          for class_id
          in annotations.class_id]
      annotates_image = mask_annotator.annotate(
          scene=image.copy(),
          detections=annotations)
      annotates_image = box_annotator.annotate(
          scene=annotates_image,
          detections=annotations,
          labels=labels)
      images.append(annotates_image)

  sv.plot_images_grid(
      images=images,
      titles=image_names,
      grid_size=SAMPLE_GRID_SIZE,
      size=SAMPLE_PLOT_SIZE)